# **模型評估**

在本 Notebook 中，您將學習：

- 什麼是模型評估？
- 如何在 ML.NET 中評估模型
- 使用交叉驗證訓練和評估模型
- 模型可解釋性
- 如何改進你的模型

## **什麼是模型評估？**

訓練是將演算法應用於歷史資料以建立準確表示該資料的模型的過程。 然後將該模型用於新資料以進行預測。

模型評估是使用指標來量化模型在資料中學習模式的效率並將這些學習應用於新的和看不見的資料的過程。

## **如何在 ML.NET 中評估模型**

### **引用 ML.NET Daily Nuget**

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json

### **安裝相關依賴**

In [1]:
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.ML.AutoML, 0.20.0-preview.22356.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

Reference packages with `using` statements

In [1]:
using System.Text.Json;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;
using static Microsoft.ML.Transforms.OneHotEncodingEstimator;

### **下載或查詢資料**
以下程式碼嘗試在幾個已知位置找到資料檔案，或者它將從已知的 GitHub 位置下載它。

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

Train Data Path: C:\dev\csharp-notebooks\machine-learning\data\taxi-fare.csv


In [1]:
var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");
var df = DataFrame.LoadCsv(trainDataPath);

載入資料後，使用 `Head` 方法預覽前五行。

In [1]:
df.Head(5)

index,vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
0,CMT,1,1,1271,3.8,CRD,17.5
1,CMT,1,1,474,1.5,CRD,8
2,CMT,1,1,637,1.4,CRD,8.5
3,CMT,1,1,181,0.6,CSH,4.5
4,CMT,1,1,661,1.1,CRD,8.5


### **初始化 MLContext**

所有 ML.NET 操作都從 [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext) 類開始。 初始化 mlContext 會建立一個新的 ML.NET 環境，該環境可以在模型建立工作流物件之間共享。 從概念上講，它類似於 Entity Framework 中的 DBContext。

In [1]:
var mlContext = new MLContext();

### 將資料拆分為訓練集、驗證集和測試集

原始資料集分為三個子集：訓練、驗證和測試。 **train** 集用於學習資料模式。 **validation** 集用於最佳化模型超引數。 **test** 集用於使用迴歸任務的評估指標來評估模型的效能。

在這種情況下，80% 的資料用於由『testFraction』引數定義的訓練。剩下的 20% 被分成兩半，用作驗證和測試集。

#### 為什麼不使用整個資料集？

儘管通常建議為您的培訓師提供更多可以從中學習的範例，但您不希望模型僅在歷史資料上表現良好。相反，您正在尋找一個模型，該模型可以從歷史資料中學習，並對新的和未見的資料進行概括或做出準確的預測。

您在訓練過程中遇到的一些常見問題是過擬合和欠擬合。欠擬合意味著選擇的訓練器不足以擬合訓練資料集，通常會導致訓練過程中的高損失和測試資料集的低分數/指標。要解決此問題，您需要選擇更強大的模型或執行更多特徵工程。過度擬合則相反，當模型對訓練資料學習得很好時就會發生這種情況。這通常會導致訓練期間的低損失度量，但測試資料集的高損失。

這些概念的一個很好的類比是為考試而學習。假設您提前知道問題和答案。學習後，您參加考試並獲得滿分。好訊息！但是，當您再次參加考試時，重新安排了問題並且措辭略有不同，您會得到較低的分數。這表明您記住了答案，實際上並沒有學習您正在測試的概念。這是過擬合的一個例子。擬合不足是相反的情況，即您獲得的學習材料不能準確地代表您在考試中所評估的內容。結果，您求助於猜測答案，因為您沒有足夠的知識來正確回答。

In [1]:
var trainTestData = mlContext.Data.TrainTestSplit(df,testFraction:0.2);
var validationTestData = mlContext.Data.TrainTestSplit(trainTestData.TestSet,testFraction:0.5);

In [1]:
var trainSet = trainTestData.TrainSet;
var validationSet = validationTestData.TrainSet;
var testSet = validationTestData.TestSet;

### 建立訓練通道

對於此資料集，應用了以下轉換：

- `OneHotEncoding` 將分類值轉換為數值
- `ReplaceMissingValues` 顧名思義就是替換任何缺失值。
- `Concatenate` 獲取所有特徵並建立一個特徵向量

AutoML 用於定義迴歸實驗，使用『fare_amount』列作為預測列或標籤列。

In [1]:
var pipeline = 
	mlContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(mlContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(mlContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }))
        .Append(mlContext.Auto().Regression(labelColumnName: "fare_amount"));

### **設定實驗**

使用 AutoML 將我們的實驗設定為使用您剛剛定義的通道進行 60 秒的訓練。

預設情況下，AutoML 使用您要最佳化的評估指標評估它訓練的模型。在這種情況下，它是 R-Squared，它是透過將實際值 `fare_amount` 與預測值 `Score` 進行比較來計算的。

評估指標高度依賴於任務。對於迴歸，一些常見的指標包括：

- 平均絕對誤差 (MAE)
- 均方誤差 (MSE)
- 均方根誤差 (RMSE)
- R平方

您的資料集和您嘗試實現的目標對您的指標選擇有很大影響。如果您的資料集中有異常值，它可能會扭曲您的預測。 MAE、MSE 和 RMSE 計算預測資料點和實際資料點之間的距離。所有這些度量對異常值都很敏感，因此如果您的資料集中有異常值，它們將顯示在您的指標中。 R-Squared 計算實際值和預測值之間的相關性。但是，隨著您新增更多資料點，您的 R-Squared 可能會繼續增加，給人一種錯誤印象，即具有高 R-Squared 值的模型具有良好的預測能力。高 R-Squared 值的結果有時可能表明過度擬合。

In [1]:
var experiment = 
	mlContext.Auto().CreateExperiment()
		.SetPipeline(pipeline)
        .SetTrainingTimeInSeconds(60)
        .SetDataset(trainSet, validationSet)
        .SetEvaluateMetric(RegressionMetric.RSquared, "fare_amount", "Score");

### **執行實驗**

In [1]:
var result = await experiment.RunAsync();

### **檢視最佳模型的評估指標**

In [1]:
$"R-Squared: {result.Metric}"

R-Squared: 0.9329530384286037

請注意，在訓練期間，評估指標是使用驗證集計算的。 要檢視您的模型在新資料上的表現如何，請根據測試集評估其效能。

首先使用訓練結果中的『模型』屬性獲取最佳模型。 然後，使用 `Transform` 方法使用模型對測試資料集進行預測。

In [1]:
ITransformer bestModel = result.Model;
var predictions = bestModel.Transform(testSet);

檢查前幾個預測（`Score` 列）並將它們與實際值（`fare_amount` 列）進行比較。 然後，計算它們之間的差異。

In [1]:
var actual = predictions.GetColumn<float>("fare_amount");
var predicted = predictions.GetColumn<float>("Score");

var compare = 
	actual
		.Zip(predicted,(actual,pred) => new {Actual=actual, Predicted=pred, Difference=actual-pred})
		.Take(5);

compare

index,Actual,Predicted,Difference
0,24.5,23.087162,1.412838
1,9.5,8.993666,0.5063343
2,4.5,4.808011,-0.30801105
3,8,7.994398,0.005601883
4,52,52.039684,-0.039684296


只需快速比較前幾個值，您就可以看到預測通常與實際金額相差幾美分。

使用 ML.NET，您不必手動計算模型的評估指標。 ML.NET 為其支援的每個機器學習任務提供了一個內建的『評估』方法。 使用迴歸任務的 `Evaluate` 方法計算測試集的評估指標，其中 `fare_amount` 列是實際值， `Score` 列是預測值。

In [1]:
var evaluationMetrics = mlContext.Regression.Evaluate(predictions,"fare_amount", "Score");

Using the `Evaluate` method not only calculates the metric you optimized for during training (R-Squared), but also all the metrics for the regression task. 

In [1]:
evaluationMetrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0.6107270253580241,6.673910566709432,2.58339129183123,6.673910534194763,0.9277130209892651


### 使用 R-Squared 評估模型

儘管您有多個指標可供選擇，但在訓練模型時，您已針對 R-Squared 進行了最佳化。 R 平方 (R2) 或確定係數將模型的預測能力表示為介於 -inf 和 1.00 之間的值。 1.00 表示完美契合，契合度可以任意差，因此得分可能為負數。 0.00 分表示模型正在猜測標籤的預期值。 負 R2 值表示擬合不遵循資料的趨勢，並且模型的效能比隨機猜測差。 這僅適用於非線性迴歸模型或約束線性迴歸。 R2 衡量實際測試資料值與預測值的接近程度。


有關其他評估指標的詳細訊息，請參閱有關如何[使用指標評估您的 ML.NET 模型](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics) 的指南。

## 使用交叉驗證訓練和評估模型

### 什麼是交叉驗證？

交叉驗證是一種訓練和模型評估技術，它將資料分成幾個分割槽並在這些分割槽上訓練多個模型。 這種技術透過保留訓練過程中的資料來提高模型的魯棒性。 除了提高未見觀察的效能外，在資料受限的環境中，它還可以成為使用較小資料集訓練模型的有效工具。

### 使用交叉驗證訓練模型

首先初始化『MLContext』。

In [1]:
var cvMLContext = new MLContext();

然後，定義您的通道。 在這種情況下，使用實際的訓練器而不是 AutoML 中的『SweepableEstimator』。

In [1]:
var cvMLPipeline = 
	cvMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(cvMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(cvMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }))
		.Append(cvMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount"));

使用 `CrossValidate` 方法開始使用定義的通道對您的資料進行訓練和評估。 預設情況下，資料分為五個子集，但您可以使用『numberOfFolds』引數將其設定為您喜歡的任何值。

In [1]:
var cvResults = cvMLContext.Regression.CrossValidate(trainSet, cvMLPipeline, labelColumnName: "fare_amount");

In [1]:
cvResults.Select(x => x.Metrics)

index,MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0,1.3080861318665928,10.437064100433819,3.230644533283385,10.437063966978595,0.8884018879298109
1,1.360165666717988,10.798742155503342,3.2861439645127146,10.798742212339462,0.8850517190709868
2,1.2518280553341605,8.216770508755435,2.8664909748253935,8.216770540499184,0.9086191464533752
3,1.3420262660438258,10.229870802662239,3.1984169213319014,10.229870780300226,0.8886983637741152
4,1.5430782453416307,10.122911443078698,3.1816523133552317,10.122911478082717,0.88835065401792


### 計算測試集評估指標

與前面的範例一樣，在完整測試集上使用『評估』方法來評估使用交叉驗證訓練的模型的效能。

In [1]:
var cvTestEvalMetrics = 
	cvResults
		.Select(fold => fold.Model.Transform(trainTestData.TestSet))
		.Select(predictions => cvMLContext.Regression.Evaluate(predictions, "fare_amount", "Score"));

In [1]:
cvTestEvalMetrics

index,MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0,1.2946164063640293,9.741100529118015,3.1210736180228134,9.741100436339362,0.8944318265394658
1,1.3517783571125017,10.697284288708751,3.270670311833455,10.697284293348838,0.8840692835505208
2,1.258738437439045,10.056973626481687,3.1712731869836897,10.056973680972417,0.8910085843878911
3,1.3321059289142654,10.119475327642087,3.1811122783771855,10.119475305281489,0.8903312286404623
4,1.5373861941022984,11.21522705311926,3.3489143096112897,11.215227048598013,0.878456132199452


## 模型可解釋性

評估指標是量化模型對新資料的預測效果的好方法。但是，一個好的評估指標不應該是您在評估模型時考慮的唯一因素。為了對您的模型及其做出的決策建立更多的信任，瞭解它如何以及為什麼做出它所做的決策非常重要。

模型在社會中變得越來越普遍，並且正在影響個人的生活。例如，假設使用機器學習模型進行醫學診斷。該模型的診斷很可能是正確的，但由於對個人健康的影響，做出錯誤診斷的風險很高。因此，所有利益相關者（患者、醫生、監管機構）都必須瞭解是什麼促使模型做出診斷，以便對其決策充滿信心。

### 全域和區域性解釋

在解釋機器學習模型時，您可以在全域和本地級別進行。

全域解釋是您在總體水平上進行概括。例如，假設您正在建立一個模型來預測出租車費用。在解釋為什麼某些票價比其他票價更貴時，您很可能會發現行程更長或時間更長的遊樂設施可能更貴。儘管這並不能準確地告訴您為什麼任何一種特定的遊樂設施都比另一種更昂貴，但在總體層面上，您可以在模型做出決策時看到哪些特徵對模型很重要。

如果您在解釋模型的決策時需要更多粒度，這就是區域性解釋的用武之地。區域性解釋允許您檢視任何單個預測，哪些特徵有助於模型的決策。例如，假設一個模型用於確定個人貸款的信用風險。給定兩個具有不同債務、收入和付款歷史的客戶，該模型確定哪個客戶更有可能償還貸款。使用本地可解釋性技術，您可以在個人層面檢查哪些特徵促成了拒絕貸款的決定。

### ML.NET 中的可解釋性技術

ML.NET 提供了兩種解釋模型的技術：

- 排列特徵重要性（PFI）
- 特徵貢獻計算（FCC）

#### 排列特徵重要性（PFI）

排列特徵重要性是一種**全域**可解釋性技術。在高層次上，它對整個資料集一次隨機打亂數據一個特徵，並計算感興趣的效能指標降低了多少。變化越大，該特徵就越重要。有關詳細訊息，請參閱 [使用排列特徵重要性解釋模型預測](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/explain-machine-learning-model-permutation-feature-importance -ml-net)

#### 特徵貢獻計算（FCC）

特徵貢獻計算是一種**本地**可解釋性技術。該技術計算每個預測的每個特徵貢獻的模型特定清單。這些貢獻可以是正面的（它們使評估指標更高）或負面的（它們使評估指標更低）。

### Explain models with Permutation Feature Importance (PFI)

#### Initialize MLContext

In [1]:
var pfiMLContext = new MLContext();

#### 定義資料準備通道

In [1]:
var pfiDataPipeline = 
	pfiMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(pfiMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(pfiMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

#### 將資料轉換應用於訓練資料

In [1]:
var pfiPreprocessedData = 
	pfiDataPipeline
		.Fit(trainSet)
		.Transform(trainSet);

#### 定義訓練器

In [1]:
var pfiTrainer = pfiMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount");

#### 使訓練器適配您的預處理資料

In [1]:
var pfiModel = pfiTrainer.Fit(pfiPreprocessedData);

#### 計算排列特徵重要性（PFI）

In [1]:
var permutationFeatureImportance =
    mlContext
        .Regression
        .PermutationFeatureImportance(pfiModel, pfiPreprocessedData, permutationCount:3, labelColumnName: "fare_amount");

#### 提取 R 平方度量

In [1]:
var pfiMetrics = 
	permutationFeatureImportance
		.Select((metric,idx) => new {idx, metric.RSquared})
		.OrderByDescending(x => Math.Abs(x.RSquared.Mean));

#### 獲取屬性名稱清單

In [1]:
var featureContributionColumn = pfiPreprocessedData.Schema.GetColumnOrNull("Features");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
featureContributionColumn.Value.GetSlotNames(ref slotNames);
var slotNameValues = slotNames.DenseValues();

#### 將 PFI 指標對映到特徵名稱

In [1]:
var featureImportance = 
	pfiMetrics
		.Zip(slotNameValues, (a,b) => new KeyValuePair<string,double>(b.ToString(),a.RSquared.Mean));

featureImportance

index,Key,Value
0,vendor_id.Bit2,-0.5103167076996584
1,vendor_id.Bit1,-0.20920573710951015
2,vendor_id.Bit0,-0.20524726245559932
3,payment_type.Bit3,-0.0013735609832817113
4,payment_type.Bit2,-0.0005371983420188927
5,payment_type.Bit1,-0.00015402329111213753
6,payment_type.Bit0,-7.256291485776185E-05
7,rate_code,-5.605438192501921E-07
8,passenger_count,0
9,trip_time_in_secs,0


### 使用特徵貢獻計算 (FCC) 解釋模型

#### MLContext 初始化

In [1]:
var fccMLContext = new MLContext();

#### 定義資料準備通道

In [1]:
var fccDataPipeline = 
	fccMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(fccMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(fccMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

#### 將資料轉換應用於訓練資料

In [1]:
var fccPreprocessedData = 
	fccDataPipeline
		.Fit(trainSet)
		.Transform(trainSet);

#### 定義訓練器

In [1]:
var fccTrainer = fccMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount");

#### 使訓練器適合您的預處理資料

In [1]:
var fccModel = fccTrainer.Fit(fccPreprocessedData);

#### 計算特徵貢獻

In [1]:
var featureContributionCalc = 
	fccMLContext.Transforms.CalculateFeatureContribution(fccModel,normalize:false)
		.Fit(fccPreprocessedData)
		.Transform(fccPreprocessedData);

#### 獲取功能名稱清單

In [1]:
var featureContributionColumn = featureContributionCalc.Schema.GetColumnOrNull("FeatureContributions");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
featureContributionColumn.Value.GetSlotNames(ref slotNames);
var slotNameValues = slotNames.DenseValues();

#### 獲取特徵貢獻值

In [1]:
var featureContributionValues = featureContributionCalc.GetColumn<float[]>("FeatureContributions");

#### 使用特徵名稱對映特徵貢獻值

In [1]:
var featureContributions = 
	featureContributionValues
		.Select(x => x.Zip(slotNameValues, (a,b) => new KeyValuePair<string,float>(b.ToString(),a)));

#### 顯示第一個預測貢獻的特徵

In [1]:
featureContributions.First()

index,Key,Value
0,vendor_id.Bit2,0
1,vendor_id.Bit1,0
2,vendor_id.Bit0,6.46816
3,payment_type.Bit3,0
4,payment_type.Bit2,-3.4585545
5,payment_type.Bit1,19.19871
6,payment_type.Bit0,10.600954
7,rate_code,-1779.6366
8,passenger_count,-1.6658905
9,trip_time_in_secs,75.95777


## **如何改進我的模型？**

模型評估是機器學習工作流程中的一個重要步驟，用於確定模型是否已準備好部署到生產中。如果您的模型不符合您的生產就緒標準，您可以嘗試一些事情來改進您的模型。這些包括：

- **重新定義問題** - 您是否正在嘗試解決正確的問題？考慮從不同的角度看待問題。
- **提供更多資料樣本** - 經驗是最好的老師。提供更多代表您的問題空間的範例有助於培訓師識別更多邊緣案例。
- **新增功能（更多上下文）** - 圍繞資料點建立上下文有助於演算法和主題專家更好地做出決策。例如，一所房子有三間臥室這一事實本身並不能很好地表明它的價格。但是，如果您新增上下文並且現在知道它位於主要大都市區以外的郊區，平均年齡為 38 歲，平均家庭收入為 80,000 美元，學校位於前 20 個百分位，那麼該演算法有更多訊息可以作為其基礎上的決定。
- **使用有意義的資料和特徵** - 更多資料和特徵（上下文）有助於提高準確性，但也會引入噪音。考慮使用置換特徵重要性 (PFI) 和特徵貢獻計算 (FCC) 來確定影響您的預測的特徵並刪除任何對您的模型沒有貢獻的特徵。
- **使用交叉驗證** - 交叉驗證可以成為訓練具有較小資料集的模型的有效工具。
- **超引數調整** - 超引數與訓練期間學習的引數一樣重要。找到合適的人是一個反覆試驗的過程。使用 AutoML 搜尋空間和全面估計器來幫助您為演算法找到正確的超引數。
- **選擇不同的演算法** - 就像超引數調整一樣，找到合適的演算法來訓練你的模型是一個反覆試驗的過程。使用 AutoML 幫助您遍歷 ML.NET 中可用的各種演算法，以幫助您選擇最佳演算法來解決您的問題。

有關詳細訊息，請參閱有關 [如何改進 ML.NET 模型] (https://docs.microsoft.com/dotnet/machine-learning/resources/improve-machine-learning-model-ml-net) 的指南。

## 進階學習

> [⏪ 上一個學習模組 - 訓練和AutoML](https://github.com/kinfeylo/csharp-notebooks/blob/main/machine-learning/03-Training%20and%20AutoML.ipynb)

### 更多實例

- [Titanic 資料集二分類](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/REF-Kaggle%20with%20Titanic%20Dataset.ipynb)  
- [使用 AutoML 進行迴歸 (計程車資料集)](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/E2E-Regression%20with%20Taxi%20Dataset.ipynb)  